In [18]:
SELECT *
INTO Sales.Orders_Tests
FROM Sales.Orders;


(73595 rows affected)

Total execution time: 00:00:00.362

We copy the Sales.Orders table and name it Sales.Orders\_Test. This is going to be our first table we practice our queries.

In [7]:
INSERT INTO Sales.Orders_Test 
(OrderID, CustomerID, SalespersonPersonID, ContactPersonID, OrderDate, ExpectedDeliveryDate, IsUndersupplyBackordered, LastEditedBy, LastEditedWhen)
VALUES (5001, 1, 2, 1, GETDATE(), DATEADD(day, 7, GETDATE()), 0, 1, GETDATE());

(1 row affected)

Total execution time: 00:00:00.004

Our first Query 

- **Purpose**: Adds a new record into the `Sales.Orders_Test` table.
- **Columns Specified**: All necessary columns are listed to match the table structure, avoiding any null constraint violations.
- **Values Provided**:
    - `OrderID`: Given a unique ID of 5001.
    - `CustomerID`, `SalespersonPersonID`, `ContactPersonID`: These foreign keys are set to valid values assuming these IDs exist.
    - `OrderDate`: Set to the current date using `GETDATE()`.
    - `ExpectedDeliveryDate`: Set to one week after the current date.
    - `IsUndersupplyBackordered`: Indicates there is no backorder, set to 0 (false).
    - `LastEditedBy` and `LastEditedWhen`: Tracks who and when the record was last updated, using an arbitrary ID and the current date/time.

In [8]:
UPDATE Sales.Orders_Test
SET ExpectedDeliveryDate = DATEADD(day, 7, ExpectedDeliveryDate)
WHERE OrderID = 5001;


(3 rows affected)

Total execution time: 00:00:00.069

Our second query

**Explanation**:

- **Purpose**: Updates the `ExpectedDeliveryDate` for a specific order.
- **DATEADD Function**: Adds an additional 7 days to the existing `ExpectedDeliveryDate`.
- **Condition**: Applies the update only to the order with `OrderID` of 5001.

In [9]:
DELETE FROM Sales.Orders_Test
WHERE OrderID = 5001;


(3 rows affected)

Total execution time: 00:00:00.026

Our third query

**Explanation**:

- **Purpose**: Removes an order from the table.
- **Condition**: Specifies that the order to be deleted has the `OrderID` of 5001, ensuring only this record is affected.

In [10]:
INSERT INTO Sales.Orders_Test (CustomerID, SalespersonPersonID, ContactPersonID, OrderDate, ExpectedDeliveryDate, IsUndersupplyBackordered, LastEditedBy, LastEditedWhen)
SELECT CustomerID, SalespersonPersonID, ContactPersonID, GETDATE(), DATEADD(day, 10, GETDATE()), IsUndersupplyBackordered, LastEditedBy, GETDATE()
FROM Sales.Orders_Test
WHERE OrderID <= 100;


: Msg 515, Level 16, State 2, Line 1
Cannot insert the value NULL into column 'OrderID', table 'WideWorldImporters.Sales.Orders_Test'; column does not allow nulls. INSERT fails.

The statement has been terminated.

Total execution time: 00:00:00.011

Our fourth query

**Explanation**:

- **Purpose**: Inserts new orders by duplicating existing orders with modifications.
- **Source for Data**: Copies data from the same table for orders with `OrderID` less than or equal to 100.
- **Modifications**: Sets the `OrderDate` to the current date and extends the `ExpectedDeliveryDate` by 10 days from now.

In [12]:
UPDATE Sales.Orders_Test
SET SalespersonPersonID = 5
WHERE OrderDate = '2014-10-01';


(76 rows affected)

Total execution time: 00:00:00.029

Our fifth query

**Explanation**:

- **Purpose**: Updates the salesperson for all orders made on a specific date.
- **Column Updated**: `SalespersonPersonID` is changed to 5 for all targeted records.
- **Condition**: Filters the update to orders placed on October 1, 2023.

In [13]:
UPDATE Sales.Orders_Test
SET IsUndersupplyBackordered = 0
WHERE IsUndersupplyBackordered = 1;


(73594 rows affected)

Total execution time: 00:00:00.158

Our sixth query

**Explanation**:

- **Purpose**: Ensures no orders are marked as backordered.
- **Column Updated**: Changes `IsUndersupplyBackordered` from true (1) to false (0).
- **Condition**: Only affects orders previously marked as backordered.

In [14]:
DELETE FROM Sales.Orders_Test
WHERE OrderDate < DATEADD(year, -1, GETDATE());


(73594 rows affected)

Total execution time: 00:00:00.135

Our seventh query

**Explanation**:

- **Purpose**: Cleans up old orders from the test table.
- **Condition**: Uses `DATEADD` to calculate the date one year ago from today and deletes records older than that date.

In [19]:
UPDATE Sales.Orders_Test
SET ExpectedDeliveryDate = DATEADD(day, 5, ExpectedDeliveryDate),
    Comments = 'Delayed delivery due to increased demand.'
WHERE ExpectedDeliveryDate > GETDATE();


(0 rows affected)

Total execution time: 00:00:00.067

Our eigth query 

**Explanation**:

- **Purpose**: Adjusts delivery dates and adds a comment regarding the delay for all future deliveries.
- **Multiple Column Updates**:
    - Increases `ExpectedDeliveryDate` by 5 days.
    - Adds a standardized comment to explain the reason for the delay.
- **Condition**: Limits updates to orders with a delivery date that is still in the futur

In [20]:
SELECT *
INTO Sales.Customers_Test
FROM Sales.Customers;


(663 rows affected)

Total execution time: 00:00:00.021

creating a copy of a second table to showcase the use of merge technique

In [21]:
MERGE INTO Sales.Customers_Test AS target
USING Sales.OldAccounts AS source
ON target.CustomerID = source.CustomerID
WHEN MATCHED THEN
    UPDATE SET target.IsActive = 0;


: Msg 208, Level 16, State 1, Line 1
Invalid object name 'Sales.OldAccounts'.

Total execution time: 00:00:00.001

Our 9th query

**MERGE INTO**: Initiates a merge operation into the `Sales.Customers_Test` table which is aliased as `target`. This table will be modified based on the conditions that follow.  

**USING**: Specifies the data source for the merge operation. `Sales.CustomerUpdates` is used as the source table, aliased as `source`. This table contains the updated data which will be compared against the target table.  

**ON**: Defines the condition for matching rows between the target and source tables. The match is made where the `CustomerID` in the target table equals the `CustomerID` in the source table.  

**WHEN MATCHED THEN**: Specifies the operation to perform when a match is found between a row in the target and a row in the source based on the condition set in the ON clause.  

**UPDATE SET**: This command updates the fields in the target table for matched records. Here, it sets:

- `PhoneNumber` to the `NewPhoneNumber` from the source.
- `FaxNumber` to the `NewFaxNumber` from the source.
- `DeliveryAddressLine1` to the `NewAddress` from the source.

**WHEN NOT MATCHED BY TARGET THEN**: Specifies what to do if no matching rows are found in the target table for the rows in the source table.

**INSERT**<span style="color: var(--vscode-foreground);">: This command inserts a new row into the target table if no matching row exists. It fills the columns with values provided from the source table, ensuring that all required fields are populated according to the source's data.</span>

<span style="color: var(--vscode-foreground);"><br></span>

<span style="color: var(--vscode-foreground);"><br></span>

In [22]:
MERGE INTO Sales.Customers_Test AS target
USING Sales.CustomerUpdates AS source
ON target.CustomerID = source.CustomerID
WHEN MATCHED THEN
    UPDATE SET 
        target.PhoneNumber = source.NewPhoneNumber,
        target.FaxNumber = source.NewFaxNumber,
        target.DeliveryAddressLine1 = source.NewAddress
WHEN NOT MATCHED BY TARGET THEN
    INSERT (CustomerID, CustomerName, PhoneNumber, FaxNumber, DeliveryAddressLine1, DeliveryMethodID, DeliveryCityID, PostalCityID, AccountOpenedDate, StandardDiscountPercentage, IsStatementSent, IsOnCreditHold, PaymentDays, LastEditedBy, ValidFrom, ValidTo)
    VALUES (source.CustomerID, source.CustomerName, source.NewPhoneNumber, source.NewFaxNumber, source.NewAddress, source.DeliveryMethodID, source.DeliveryCityID, source.PostalCityID, source.AccountOpenedDate, source.StandardDiscountPercentage, source.IsStatementSent, source.IsOnCreditHold, source.PaymentDays, source.LastEditedBy, source.ValidFrom, source.ValidTo);


: Msg 208, Level 16, State 1, Line 1
Invalid object name 'Sales.CustomerUpdates'.

Total execution time: 00:00:00.001

Our 10th query

 this starts a merge operation targeting the `Sales.Customers_Test` table, aliasing it as `target`.  

Sets `Sales.OldAccounts` as the source table for the merge, aliasing it as `source`. This table contains identifiers of customer accounts that are considered old or inactive.  

The match condition between the target and source tables is based on `CustomerID`.  

Specifies the operation to execute when there's a matching row found between the target and source tables based on the `CustomerID`.  

**UPDATE SET**: Updates the `IsActive` field in the target table to `0` for matched rows, marking these customer accounts as inactive.